# Week 1: Lab Solutions - Multi-Armed Bandits


This notebook contains complete solutions with detailed explanations.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple

np.random.seed(42)

## Part 1: Bandit Environment - Solution

**Key Concepts:**
- Each arm has a Gaussian (normal) reward distribution
- The mean determines the expected reward, std controls variance
- We use `np.random.randn()` for standard normal and scale it

In [ ]:
class MultiArmedBandit:
    """
    Multi-Armed Bandit environment.
    
    Each arm has a Gaussian reward distribution with a specific mean and standard deviation.
    """
    
    def __init__(self, n_arms: int, means: List[float] = None, std: float = 1.0):
        """
        Initialize the bandit.
        
        Args:
            n_arms: Number of arms
            means: List of mean rewards for each arm. If None, randomly generate.
            std: Standard deviation of reward distributions
        """
        self.n_arms = n_arms
        self.std = std
        
        if means is None:
            # Generate random means from a standard normal distribution
            # This ensures diversity in arm quality
            self.means = np.random.randn(n_arms)
        else:
            self.means = np.array(means)
        
        # Track which arm is optimal (highest expected reward)
        self.optimal_arm = np.argmax(self.means)
    
    def pull(self, arm: int) -> float:
        """
        Pull an arm and receive a reward.
        
        The reward is sampled from N(mean[arm], std^2)
        
        Args:
            arm: Index of the arm to pull (0 to n_arms-1)
            
        Returns:
            reward: Reward sampled from the arm's distribution
        """
        # Sample from N(mean, std^2) = mean + std * N(0, 1)
        reward = self.means[arm] + self.std * np.random.randn()
        return reward
    
    def get_optimal_value(self) -> float:
        """Return the expected reward of the optimal arm."""
        return self.means[self.optimal_arm]


# Test the implementation
bandit = MultiArmedBandit(n_arms=5, means=[0.5, 1.2, 0.8, 2.1, 1.5])
print(f"Bandit means: {bandit.means}")
print(f"Optimal arm: {bandit.optimal_arm} with mean {bandit.get_optimal_value():.2f}")
print(f"\nSample rewards from arm {bandit.optimal_arm}: ", [f"{bandit.pull(bandit.optimal_arm):.2f}" for _ in range(5)])

### Explanation:

**Why Gaussian distributions?**
- They're realistic: many real-world rewards have natural variation around a mean
- They're unbounded: rewards can theoretically be any value
- They're mathematically convenient

**Random mean initialization:**
- Ensures different problem instances have different optimal arms
- Tests that our agent can find the best arm regardless of which one it is

## Part 2: Epsilon-Greedy Agent - Solution

**Key Concepts:**
- **Exploration (ε):** Try random actions to learn about them
- **Exploitation (1-ε):** Use current knowledge to get best reward
- **Incremental update:** Efficiently compute running average without storing all rewards

In [ ]:
class EpsilonGreedyAgent:
    """
    Agent using epsilon-greedy action selection.
    """
    
    def __init__(self, n_arms: int, epsilon: float = 0.1, initial_value: float = 0.0):
        """
        Initialize the agent.
        
        Args:
            n_arms: Number of arms
            epsilon: Exploration probability
            initial_value: Initial Q-value estimates for all arms
        """
        self.n_arms = n_arms
        self.epsilon = epsilon
        
        # Q-values: estimated value of each arm
        self.Q = np.ones(n_arms) * initial_value
        
        # Count how many times each arm was pulled
        self.N = np.zeros(n_arms)
    
    def select_action(self) -> int:
        """
        Select an action using epsilon-greedy strategy.
        
        With probability epsilon: explore (random action)
        With probability 1-epsilon: exploit (best known action)
        
        Returns:
            action: Selected arm index
        """
        if np.random.random() < self.epsilon:
            # Explore: choose random arm
            action = np.random.randint(self.n_arms)
        else:
            # Exploit: choose arm with highest Q-value
            action = np.argmax(self.Q)
        
        return action
    
    def update(self, action: int, reward: float):
        """
        Update Q-value estimate after observing a reward.
        
        Uses incremental mean formula:
        NewEstimate = OldEstimate + StepSize * (Target - OldEstimate)
        
        Where StepSize = 1/N gives us the sample average.
        
        Args:
            action: The arm that was pulled
            reward: The observed reward
        """
        # Increment count for this arm
        self.N[action] += 1
        
        # Update Q-value using incremental mean formula
        # Q_new = Q_old + (1/N) * (reward - Q_old)
        # This is equivalent to: Q_new = (Q_old * (N-1) + reward) / N
        # but more numerically stable and efficient
        self.Q[action] += (reward - self.Q[action]) / self.N[action]


# Test the implementation
agent = EpsilonGreedyAgent(n_arms=5, epsilon=0.1)
print(f"Initial Q-values: {agent.Q}")

# Simulate pulls from a fixed arm to verify learning
test_arm = 2
true_mean = 1.5
for _ in range(100):
    reward = true_mean + np.random.randn()  # N(1.5, 1)
    agent.update(test_arm, reward)

print(f"\nAfter 100 pulls of arm {test_arm} (true mean = {true_mean}):")
print(f"Learned Q-value for arm {test_arm}: {agent.Q[test_arm]:.3f}")
print(f"Number of pulls: {agent.N[test_arm]:.0f}")

### Explanation:

**Why the incremental update formula?**

Instead of storing all rewards and computing:
```python
Q[a] = mean(all_rewards_for_arm_a)
```

We use:
```python
Q[a] = Q[a] + (reward - Q[a]) / N[a]
```

This is:
1. **Memory efficient:** Only store one value per arm
2. **Computationally efficient:** O(1) update instead of O(N)
3. **Mathematically equivalent:** Gives the same sample average

**Understanding the update:**
- If `reward > Q[a]`, we increase Q[a]
- If `reward < Q[a]`, we decrease Q[a]
- The step size `1/N[a]` decreases over time → estimates stabilize

**Epsilon value choice:**
- ε = 0.1 means 10% exploration, 90% exploitation
- This is a common default that works well in many problems
- Too small ε → might miss the best arm
- Too large ε → waste time on suboptimal arms

## Part 3: UCB Agent - Solution

**Key Concepts:**
- **Optimism in the face of uncertainty:** Arms we've tried less are more uncertain
- **Bonus term:** $c\sqrt{\frac{\ln t}{N_t(a)}}$ gives higher priority to less-tried arms
- **Automatic exploration:** No need to manually set exploration rate

In [ ]:
class UCBAgent:
    """
    Agent using Upper Confidence Bound (UCB) action selection.
    """
    
    def __init__(self, n_arms: int, c: float = 2.0, initial_value: float = 0.0):
        """
        Initialize the agent.
        
        Args:
            n_arms: Number of arms
            c: Exploration parameter (typically 2.0)
            initial_value: Initial Q-value estimates
        """
        self.n_arms = n_arms
        self.c = c
        
        self.Q = np.ones(n_arms) * initial_value
        self.N = np.zeros(n_arms)
        self.t = 0  # Total time steps
    
    def select_action(self) -> int:
        """
        Select an action using UCB.
        
        UCB formula: A_t = argmax[Q(a) + c * sqrt(ln(t) / N(a))]
        
        Returns:
            action: Selected arm index
        """
        self.t += 1
        
        # For arms that haven't been pulled yet, give them infinite value
        # This ensures all arms are tried at least once
        if np.any(self.N == 0):
            # Return the first untried arm
            action = np.argmax(self.N == 0)
        else:
            # Calculate UCB values for all arms
            # UCB = Q + c * sqrt(ln(t) / N)
            ucb_values = self.Q + self.c * np.sqrt(np.log(self.t) / self.N)
            action = np.argmax(ucb_values)
        
        return action
    
    def update(self, action: int, reward: float):
        """
        Update Q-value estimate.
        
        Same incremental update as epsilon-greedy.
        
        Args:
            action: The arm that was pulled
            reward: The observed reward
        """
        self.N[action] += 1
        self.Q[action] += (reward - self.Q[action]) / self.N[action]


# Test the implementation
agent = UCBAgent(n_arms=5, c=2.0)
bandit = MultiArmedBandit(n_arms=5, means=[0.5, 1.2, 0.8, 2.1, 1.5])

print("Initial state:")
print(f"Q-values: {agent.Q}")
print(f"\nFirst {agent.n_arms} actions (should try each arm once):")

for step in range(5):
    action = agent.select_action()
    reward = bandit.pull(action)
    agent.update(action, reward)
    print(f"Step {step+1}: Selected arm {action}, reward = {reward:.2f}")

print(f"\nAfter initial exploration:")
print(f"Q-values: {agent.Q}")
print(f"Arm counts: {agent.N}")

# Continue for more steps
for _ in range(20):
    action = agent.select_action()
    reward = bandit.pull(action)
    agent.update(action, reward)

print(f"\nAfter 25 total steps:")
print(f"Q-values: {agent.Q}")
print(f"True means: {bandit.means}")
print(f"Arm counts: {agent.N}")
print(f"\nMost selected arm: {np.argmax(agent.N)} (optimal is {bandit.optimal_arm})")

### Explanation:

**Understanding the UCB formula:**

$$UCB(a) = Q(a) + c\sqrt{\frac{\ln t}{N(a)}}$$

- **$Q(a)$:** Current estimate (exploitation term)
- **$c$:** Exploration parameter (controls exploration degree)
- **$\sqrt{\frac{\ln t}{N(a)}}$:** Uncertainty bonus (exploration term)

**Why this works:**
1. The bonus term is large when $N(a)$ is small (arm rarely tried)
2. The bonus term grows with $t$ (more total experience → more confidence to explore)
3. As we pull an arm more, $N(a)$ increases → bonus decreases → focus on exploitation

**Key advantages over epsilon-greedy:**
- **Adaptive exploration:** Automatically reduces exploration over time
- **Directed exploration:** Focuses on uncertain arms, not random arms
- **Theoretical guarantees:** Provably finds optimal arm with logarithmic regret

**Initial exploration:**
- We explicitly check if any arm has $N(a) = 0$
- This avoids division by zero
- Ensures all arms are tried at least once before UCB selection kicks in

## Part 4: Experiment Runner - Solution

In [ ]:
def run_experiment(agent_class, agent_params: dict, bandit: MultiArmedBandit, 
                   n_steps: int = 1000) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Run a single experiment with a given agent and bandit.
    
    Args:
        agent_class: The agent class to use (EpsilonGreedyAgent or UCBAgent)
        agent_params: Dictionary of parameters to pass to agent constructor
        bandit: The bandit environment
        n_steps: Number of steps to run
        
    Returns:
        rewards: Array of rewards obtained at each step
        optimal_actions: Binary array indicating if optimal action was taken
        q_values_final: Final Q-value estimates
    """
    # Create the agent
    agent = agent_class(**agent_params)
    
    # Arrays to store results
    rewards = np.zeros(n_steps)
    optimal_actions = np.zeros(n_steps)
    
    # Run the experiment
    for step in range(n_steps):
        # Select action
        action = agent.select_action()
        
        # Pull the bandit arm
        reward = bandit.pull(action)
        
        # Update the agent
        agent.update(action, reward)
        
        # Record metrics
        rewards[step] = reward
        optimal_actions[step] = 1 if action == bandit.optimal_arm else 0
    
    return rewards, optimal_actions, agent.Q


def run_multiple_experiments(agent_class, agent_params: dict, n_experiments: int = 100, 
                            n_steps: int = 1000, n_arms: int = 10) -> Tuple[np.ndarray, np.ndarray]:
    """
    Run multiple experiments and average the results.
    
    Args:
        agent_class: The agent class to use
        agent_params: Parameters for the agent
        n_experiments: Number of experiments to run
        n_steps: Steps per experiment
        n_arms: Number of arms in the bandit
        
    Returns:
        avg_rewards: Average rewards across experiments
        avg_optimal: Average percentage of optimal actions
    """
    all_rewards = np.zeros((n_experiments, n_steps))
    all_optimal = np.zeros((n_experiments, n_steps))
    
    for i in range(n_experiments):
        # Create a new bandit for each experiment
        # This tests robustness across different problem instances
        bandit = MultiArmedBandit(n_arms=n_arms)
        
        rewards, optimal_actions, _ = run_experiment(agent_class, agent_params, bandit, n_steps)
        
        all_rewards[i] = rewards
        all_optimal[i] = optimal_actions
    
    return np.mean(all_rewards, axis=0), np.mean(all_optimal, axis=0)


# Test with a simple case
test_bandit = MultiArmedBandit(n_arms=5, means=[0.5, 1.2, 0.8, 2.1, 1.5])
rewards, optimal, final_q = run_experiment(
    EpsilonGreedyAgent, 
    {'n_arms': 5, 'epsilon': 0.1}, 
    test_bandit, 
    n_steps=1000
)

print(f"Test run completed: {len(rewards)} rewards collected")
print(f"Average reward: {np.mean(rewards):.3f}")
print(f"Optimal action rate: {np.mean(optimal)*100:.1f}%")
print(f"\nFinal Q-values: {final_q}")
print(f"True means: {test_bandit.means}")

### Explanation:

**Why run multiple experiments?**
- Each experiment has randomness (in bandit rewards and agent's random exploration)
- Averaging removes this noise and shows true performance
- We use different bandit instances to test generalization

**Metrics we track:**
1. **Rewards:** Direct measure of performance
2. **Optimal action rate:** How often we pick the best arm (regardless of reward)
3. **Final Q-values:** Shows if agent learned true arm values

## Part 5: Compare Epsilon-Greedy with Different Epsilon Values - Solution

In [ ]:
# Run experiments for different epsilon values
epsilon_values = [0, 0.01, 0.1, 0.3]
n_experiments = 100
n_steps = 1000

results = {}

print("Running experiments for different epsilon values...")
for eps in epsilon_values:
    print(f"  ε = {eps}...")
    avg_rewards, avg_optimal = run_multiple_experiments(
        EpsilonGreedyAgent,
        {'n_arms': 10, 'epsilon': eps},
        n_experiments=n_experiments,
        n_steps=n_steps,
        n_arms=10
    )
    results[eps] = {'rewards': avg_rewards, 'optimal': avg_optimal}

print("Done!\n")

# Plotting
plt.figure(figsize=(14, 5))

# Plot 1: Average reward over time
plt.subplot(1, 2, 1)
for eps in epsilon_values:
    plt.plot(results[eps]['rewards'], label=f'ε = {eps}', alpha=0.8)

plt.xlabel('Steps', fontsize=12)
plt.ylabel('Average Reward', fontsize=12)
plt.title('Average Reward vs Steps (Different Epsilon Values)', fontsize=13)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

# Plot 2: Optimal action percentage
plt.subplot(1, 2, 2)
for eps in epsilon_values:
    plt.plot(results[eps]['optimal'] * 100, label=f'ε = {eps}', alpha=0.8)

plt.xlabel('Steps', fontsize=12)
plt.ylabel('% Optimal Action', fontsize=12)
plt.title('Optimal Action Selection Rate (Different Epsilon Values)', fontsize=13)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print summary statistics
print("\nSummary Statistics (last 100 steps):")
print("-" * 60)
for eps in epsilon_values:
    final_reward = np.mean(results[eps]['rewards'][-100:])
    final_optimal = np.mean(results[eps]['optimal'][-100:]) * 100
    print(f"ε = {eps:4.2f}  |  Avg Reward: {final_reward:5.3f}  |  Optimal: {final_optimal:5.1f}%")

### Analysis of Results:

**Expected observations:**

1. **ε = 0 (Pure Greedy):**
   - **Problem:** Gets stuck on first arm that seems good
   - **Initial performance:** Can be good if lucky with initial pulls
   - **Long-term:** Usually converges to suboptimal arm
   - **Optimal action rate:** Low, often around 10-30%

2. **ε = 0.01 (Very low exploration):**
   - **Initial learning:** Slow, as it rarely explores
   - **Long-term:** Better than greedy, but may still miss optimal
   - **Optimal action rate:** Moderate, around 40-60%

3. **ε = 0.1 (Balanced):**
   - **Initial learning:** Fast discovery of good arms
   - **Long-term:** Strong performance
   - **Optimal action rate:** High, around 70-85%
   - **Best overall** for this problem

4. **ε = 0.3 (High exploration):**
   - **Initial learning:** Fastest to try all arms
   - **Long-term:** Wastes time on suboptimal arms
   - **Optimal action rate:** Limited by exploration (max ~70% since 30% random)
   - **Total reward:** Lower than ε=0.1 despite finding optimal arm

**Key insight:** There's a sweet spot for ε (usually 0.05-0.1) that balances exploration and exploitation optimally.

## Part 6: Compare Epsilon-Greedy vs UCB - Solution

In [ ]:
print("Running comparison experiments...")
print("  Epsilon-Greedy (ε=0.1)...")

# Run epsilon-greedy
rewards_egreedy, optimal_egreedy = run_multiple_experiments(
    EpsilonGreedyAgent,
    {'n_arms': 10, 'epsilon': 0.1},
    n_experiments=100,
    n_steps=1000,
    n_arms=10
)

print("  UCB (c=2.0)...")

# Run UCB
rewards_ucb, optimal_ucb = run_multiple_experiments(
    UCBAgent,
    {'n_arms': 10, 'c': 2.0},
    n_experiments=100,
    n_steps=1000,
    n_arms=10
)

print("Done!\n")

# Plotting
plt.figure(figsize=(14, 5))

# Plot 1: Average reward
plt.subplot(1, 2, 1)
plt.plot(rewards_egreedy, label='Epsilon-Greedy (ε=0.1)', linewidth=2, alpha=0.8)
plt.plot(rewards_ucb, label='UCB (c=2.0)', linewidth=2, alpha=0.8)
plt.xlabel('Steps', fontsize=12)
plt.ylabel('Average Reward', fontsize=12)
plt.title('Epsilon-Greedy vs UCB: Average Reward', fontsize=13)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)

# Plot 2: Optimal action rate
plt.subplot(1, 2, 2)
plt.plot(optimal_egreedy * 100, label='Epsilon-Greedy (ε=0.1)', linewidth=2, alpha=0.8)
plt.plot(optimal_ucb * 100, label='UCB (c=2.0)', linewidth=2, alpha=0.8)
plt.xlabel('Steps', fontsize=12)
plt.ylabel('% Optimal Action', fontsize=12)
plt.title('Epsilon-Greedy vs UCB: Optimal Action Rate', fontsize=13)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print final statistics
print("\nFinal Performance (last 100 steps average):")
print("-" * 60)
print(f"Epsilon-Greedy (ε=0.1):")
print(f"  Average Reward:      {np.mean(rewards_egreedy[-100:]):.4f}")
print(f"  Optimal Action Rate: {np.mean(optimal_egreedy[-100:])*100:.2f}%")
print(f"\nUCB (c=2.0):")
print(f"  Average Reward:      {np.mean(rewards_ucb[-100:]):.4f}")
print(f"  Optimal Action Rate: {np.mean(optimal_ucb[-100:])*100:.2f}%")

# Additional analysis: convergence speed
print("\n" + "-" * 60)
print("Convergence Analysis:")
print("-" * 60)

# Find when each algorithm reaches 50% optimal action rate
threshold = 0.5
egreedy_50 = np.argmax(optimal_egreedy > threshold)
ucb_50 = np.argmax(optimal_ucb > threshold)

print(f"Steps to reach 50% optimal action rate:")
print(f"  Epsilon-Greedy: {egreedy_50} steps")
print(f"  UCB:           {ucb_50} steps")

# Cumulative regret (difference from optimal)
# Note: This is approximate since we don't know the true optimal value
# We'll use the best average reward seen as a proxy
best_reward = max(np.max(rewards_egreedy), np.max(rewards_ucb))
regret_egreedy = np.cumsum(best_reward - rewards_egreedy)
regret_ucb = np.cumsum(best_reward - rewards_ucb)

print(f"\nCumulative Regret (lower is better):")
print(f"  Epsilon-Greedy: {regret_egreedy[-1]:.2f}")
print(f"  UCB:           {regret_ucb[-1]:.2f}")

### Deep Analysis: Epsilon-Greedy vs UCB

**1. Initial Learning Phase (Steps 0-100):**

- **UCB typically converges faster initially**
  - Systematically explores all arms first (N=0 handling)
  - Directed exploration based on uncertainty
  - Quickly identifies promising arms

- **Epsilon-Greedy starts slower**
  - Random exploration means some arms tried multiple times before others
  - Can get "unlucky" and focus on suboptimal arms early
  - More variable initial performance

**2. Mid-term Performance (Steps 100-500):**

- **UCB's advantage becomes clear**
  - Exploration naturally decreases as uncertainty reduces
  - Focuses more on promising arms
  - Better cumulative reward

- **Epsilon-Greedy catches up**
  - More samples → better estimates
  - But still wastes 10% of actions on random exploration

**3. Long-term Performance (Steps 500-1000):**

- **UCB maintains edge**
  - Adaptive exploration means almost always picking optimal arm
  - Can reach >95% optimal action rate
  - Lower cumulative regret

- **Epsilon-Greedy plateaus**
  - Limited to ~90% optimal (10% random)
  - Continues wasting actions on exploration
  - But simple and reliable

**4. Key Trade-offs:**

| Aspect | Epsilon-Greedy | UCB |
|--------|---------------|-----|
| **Simplicity** | Very simple | More complex |
| **Hyperparameters** | One (ε), easy to tune | One (c), less intuitive |
| **Computational cost** | Very low | Slightly higher (sqrt, log) |
| **Theoretical guarantees** | Weak | Strong (logarithmic regret) |
| **Performance** | Good | Better |
| **Adaptivity** | Fixed exploration | Adaptive exploration |
| **Robustness** | Very robust | Sensitive to c choice |

**5. When to use each:**

**Use Epsilon-Greedy when:**
- Simplicity is paramount
- Computational resources are very limited
- You need interpretable behavior
- Problem is non-stationary (changing over time)
- Quick prototyping

**Use UCB when:**
- Maximizing long-term reward is critical
- Problem is stationary (fixed arm distributions)
- You want theoretical guarantees
- You can afford slightly more computation
- Optimal performance matters more than simplicity

## Part 7: Analysis Questions - Sample Answers

### 1. Effect of epsilon:

**ε = 0 (Pure Greedy):**
- Gets stuck on the first arm that appears good
- No exploration means it can't discover better arms
- Performance is highly dependent on initial random pulls
- Can work if the initial estimate happens to be correct, but usually fails
- Example: If first pull of arm 3 gives high reward by chance, agent keeps pulling arm 3 even if arm 7 is actually better

**ε = 0.3 (Too Large):**
- Explores too much, wasting 30% of actions on random pulls
- Finds optimal arm quickly but doesn't exploit it enough
- Average reward is limited because of constant random exploration
- Optimal action rate can never exceed 70% (since 30% is random)
- Good for initial learning, bad for long-term performance

**Best ε:**
- For this problem, ε = 0.1 works best
- Provides enough exploration to find optimal arm
- Doesn't waste too many actions on suboptimal arms
- Achieves good balance between learning and earning
- General rule: Start with ε ∈ [0.05, 0.15] and adjust based on problem

### 2. Epsilon-Greedy vs UCB:

**Initial Convergence:**
- UCB converges faster in the first 50-100 steps
- Reason: UCB systematically tries each arm first, then focuses on promising ones
- Epsilon-greedy's random exploration is less efficient initially
- UCB's directed exploration (based on uncertainty) is more effective

**Long-term Performance:**
- UCB achieves better long-term performance
- Reaches higher optimal action rates (>95% vs ~90%)
- Lower cumulative regret
- Epsilon-greedy is limited by its fixed exploration rate
- UCB's adaptive exploration means it "learns to exploit" better over time

**When to Prefer UCB:**
- When you need optimal performance and have many steps
- Stationary problems (arm distributions don't change)
- When theoretical guarantees matter
- When you can afford slight computational overhead
- Applications: Clinical trials, A/B testing with long run times, resource allocation

### 3. Trade-offs:

**Epsilon-Greedy Advantages:**
- **Simplicity:** Only 3 lines of code for action selection
- **Intuitive:** Everyone understands "try random things 10% of the time"
- **Robust:** Works reasonably well across many problems
- **Easy tuning:** ε is interpretable and easy to adjust
- **Computational:** Extremely fast (just argmax and random)
- **Non-stationary:** Fixed exploration helps track changing environments

**UCB Advantages:**
- **Performance:** Provably optimal with logarithmic regret
- **Adaptive:** Automatically reduces exploration over time
- **Intelligent exploration:** Explores uncertain arms, not random arms
- **No waste:** Eventually almost always picks optimal arm
- **Theoretical guarantees:** Mathematically proven to work

**Scenarios for Epsilon-Greedy:**
1. **Quick prototyping:** Need to test RL quickly
2. **Teaching/Learning:** Introducing RL concepts
3. **Non-stationary environments:** User preferences change over time
4. **Embedded systems:** Limited computation (no sqrt/log operations)
5. **Short-term problems:** Only 100-200 steps, simplicity matters more
6. **Unknown problem characteristics:** Safe default that usually works

**Real-world example:** A recommendation system for news articles
- News relevance changes quickly (non-stationary)
- Need fast decisions (millions of users)
- Epsilon-greedy is better: simple, fast, handles changing interests
- UCB's stationarity assumption is violated, and computation matters

## Bonus Challenge Solutions

### 1. Decaying Epsilon

In [ ]:
class DecayingEpsilonGreedyAgent:
    """Epsilon-greedy with decaying exploration rate."""
    
    def __init__(self, n_arms: int, epsilon_start: float = 1.0, 
                 epsilon_end: float = 0.01, decay_rate: float = 0.995):
        self.n_arms = n_arms
        self.epsilon = epsilon_start
        self.epsilon_end = epsilon_end
        self.decay_rate = decay_rate
        
        self.Q = np.zeros(n_arms)
        self.N = np.zeros(n_arms)
    
    def select_action(self) -> int:
        if np.random.random() < self.epsilon:
            action = np.random.randint(self.n_arms)
        else:
            action = np.argmax(self.Q)
        
        # Decay epsilon after each action
        self.epsilon = max(self.epsilon_end, self.epsilon * self.decay_rate)
        return action
    
    def update(self, action: int, reward: float):
        self.N[action] += 1
        self.Q[action] += (reward - self.Q[action]) / self.N[action]

print("Testing Decaying Epsilon-Greedy:")
agent = DecayingEpsilonGreedyAgent(n_arms=10, epsilon_start=1.0, epsilon_end=0.01, decay_rate=0.995)
epsilons = []
for _ in range(1000):
    epsilons.append(agent.epsilon)
    action = agent.select_action()

plt.figure(figsize=(10, 4))
plt.plot(epsilons)
plt.xlabel('Steps')
plt.ylabel('Epsilon')
plt.title('Epsilon Decay Over Time')
plt.grid(True, alpha=0.3)
plt.show()

print(f"Starting epsilon: {epsilons[0]:.3f}")
print(f"Final epsilon: {epsilons[-1]:.3f}")

**Why Decaying Epsilon Works:**
- Early on: High exploration to discover good arms
- Later: Low exploration to maximize rewards
- Mimics UCB's adaptive exploration behavior
- Common in deep RL (e.g., DQN uses this)

### 2. Optimistic Initial Values

In [ ]:
# Compare pessimistic (Q=0) vs optimistic (Q=5) initialization
print("Comparing different initial Q-values...")

# Pessimistic
rewards_pessimistic, optimal_pessimistic = run_multiple_experiments(
    EpsilonGreedyAgent,
    {'n_arms': 10, 'epsilon': 0, 'initial_value': 0.0},  # Greedy with Q=0
    n_experiments=50,
    n_steps=1000
)

# Optimistic
rewards_optimistic, optimal_optimistic = run_multiple_experiments(
    EpsilonGreedyAgent,
    {'n_arms': 10, 'epsilon': 0, 'initial_value': 5.0},  # Greedy with Q=5
    n_experiments=50,
    n_steps=1000
)

plt.figure(figsize=(12, 4))
plt.plot(optimal_pessimistic * 100, label='Pessimistic (Q=0)', alpha=0.7)
plt.plot(optimal_optimistic * 100, label='Optimistic (Q=5)', alpha=0.7)
plt.xlabel('Steps')
plt.ylabel('% Optimal Action')
plt.title('Effect of Initial Q-values on Greedy Agent')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Final optimal rate (Q=0): {np.mean(optimal_pessimistic[-100:])*100:.1f}%")
print(f"Final optimal rate (Q=5): {np.mean(optimal_optimistic[-100:])*100:.1f}%")

**Optimistic Initial Values Insight:**
- Setting Q = 5 (optimistic) encourages exploration even with ε = 0
- Agent thinks all arms are great initially
- When it tries an arm and gets reward < 5, it's "disappointed"
- Tries other arms looking for the "5 reward" it expects
- Simple way to encourage exploration without explicit randomness
- Works well for stationary problems
- Problem: Need to know a good optimistic value (problem-dependent)

## Summary: Key Takeaways from Week 1

1. **RL is about learning through interaction**
   - No labels, only reward signals
   - Must balance exploration and exploitation

2. **Multi-Armed Bandits are fundamental**
   - Simplest RL problem (no state transitions)
   - Core challenge: exploration vs exploitation
   - Foundation for more complex RL

3. **Multiple valid strategies exist**
   - Epsilon-greedy: Simple, robust, good baseline
   - UCB: Principled, adaptive, optimal guarantees
   - No single "best" method for all problems

4. **Implementation matters**
   - Incremental updates are efficient
   - Proper testing requires multiple experiments
   - Visualization reveals algorithm behavior

5. **Next steps: Adding state**
   - Bandits have no state (each action independent)
   - Next week: MDPs add state transitions
   - Actions affect future states and rewards
   - Need to plan ahead, not just maximize immediate reward